In [1]:
def debug_syllables(text):
    import unicodedata
    text = unicodedata.normalize("NFC", text)

    HALANT = '್'
    ANUSVARA = 'ಂ'
    VISARGA = 'ಃ'

    LAGHU_SIGNS = ['ಿ', 'ು', 'ೃ', 'ೆ', 'ೊ']
    GURU_SIGNS = ['ಾ', 'ೀ', 'ೂ', 'ೇ', 'ೈ', 'ೋ', 'ೌ']

    LAGHU_VOWELS = ['ಅ', 'ಇ', 'ಉ', 'ಋ', 'ಎ', 'ಒ']
    GURU_VOWELS = ['ಆ', 'ಈ', 'ಊ', 'ಏ', 'ಐ', 'ಓ', 'ಔ']

    KANNADA_CONSONANTS = [chr(i) for i in range(0x0C95, 0x0CB9 + 1)]

    i = 0
    total = 0
    syllables = []

    while i < len(text):
        char = text[i]

        if char == " ":
            i += 1
            continue

        # Independent vowels
        if char in LAGHU_VOWELS:
            syllables.append((char, 1))
            i += 1
            continue

        if char in GURU_VOWELS:
            syllables.append((char, 2))
            i += 1
            continue

        # Consonant syllable
        if char in KANNADA_CONSONANTS:
            syllable = char
            weight = 1
            i += 1

            # Attach halant clusters before vowel
            while i < len(text) - 1 and text[i] == HALANT:
                syllable += text[i] + text[i+1]
                i += 2
                weight = 2

            # Attach vowel sign
            if i < len(text) and text[i] in LAGHU_SIGNS:
                syllable += text[i]
                weight = 1
                i += 1

            elif i < len(text) and text[i] in GURU_SIGNS:
                syllable += text[i]
                weight = 2
                i += 1

            # Anusvara / Visarga
            if i < len(text) and text[i] in [ANUSVARA, VISARGA]:
                syllable += text[i]
                weight = 2
                i += 1

            # Attach vowel sign
            if i < len(text) and text[i] in LAGHU_SIGNS:
                syllable += text[i]
                weight = 1
                i += 1

            elif i < len(text) and text[i] in GURU_SIGNS:
                syllable += text[i]
                weight = 2
                i += 1

            # Anusvara / Visarga
            # Attach ALL trailing halant clusters (like ಳ್ಳಲ್ case)
            while i < len(text) - 1 and text[i] in KANNADA_CONSONANTS and text[i+1] == HALANT:
                syllable += text[i] + text[i+1]
                weight = 2
                i += 2


            # Attach final closing consonant (like ಯೊಳ್ case)
            if i < len(text) - 1 and text[i] in KANNADA_CONSONANTS and text[i+1] == HALANT:
                syllable += text[i] + text[i+1]
                weight = 2
                i += 2

            # Ignore pure trailing consonant clusters like ಳಲ್
            if not (len(syllable) <= 2 and syllable.endswith(HALANT)):
                syllables.append((syllable, weight))

            continue

        i += 1

    # ---------------------------
    # Prosody Rule Adjustment
    # Laghu becomes Guru if next syllable begins with cluster
    # ---------------------------

    adjusted = []

    for idx, (syll, wt) in enumerate(syllables):
        if wt == 1 and idx + 1 < len(syllables):
            next_syll = syllables[idx + 1][0]

            # If next syllable starts with consonant cluster (has halant inside first two chars)
            if HALANT in next_syll[:2]:
                wt = 2

        adjusted.append((syll, wt))

    # Print results
    print("\n--- SYLLABLE DEBUG ---")
    for syll, wt in adjusted:
        print(f"{syll} → {wt}")
        total += wt

    print("TOTAL:", total)
    return total


In [2]:
def check_kanda_padya():
    print("Enter 4 lines of the poem:\n")

    lines = []
    for i in range(4):
        line = input(f"Line {i+1}: ")
        lines.append(line)

    print("\n--- MATRA COUNT ---")

    counts = []
    for idx, line in enumerate(lines):
        matra = debug_syllables(line)
        counts.append(matra)
        print(f"Line {idx+1}: {matra}")

    print("\n--- RESULT ---")

    if counts == [12, 20, 12, 20]:
        print("This is a Kanda Padya!")
    else:
        print("This is NOT a Kanda Padya.")
        print("Expected pattern: 12-20-12-20")


In [3]:
check_kanda_padya()

Enter 4 lines of the poem:



Line 1:  ಕಾವೇರಿಯಿಂದ ಮಾಗೋ
Line 2:  ದಾವರ ವರ ಮಿರ್ದನಾಡದಾ ಕನ್ನಡದೊಳ್
Line 3:  ಭಾವಿಸಿದ ಜನಪದಂವಸು
Line 4:  ಧಾವಳಯವಿಲೀನ ವಿಶದ ವಿಷಯವಿಶೇಷಂ



--- MATRA COUNT ---

--- SYLLABLE DEBUG ---
ಕಾ → 2
ವೇ → 2
ರಿ → 1
ಯಿಂ → 2
ದ → 1
ಮಾ → 2
ಗೋ → 2
TOTAL: 12
Line 1: 12

--- SYLLABLE DEBUG ---
ದಾ → 2
ವ → 1
ರ → 1
ವ → 1
ರ → 1
ಮಿರ್ → 2
ದ → 1
ನಾ → 2
ಡ → 1
ದಾ → 2
ಕನ್ → 2
ನ → 1
ಡ → 1
ದೊಳ್ → 2
TOTAL: 20
Line 2: 20

--- SYLLABLE DEBUG ---
ಭಾ → 2
ವಿ → 1
ಸಿ → 1
ದ → 1
ಜ → 1
ನ → 1
ಪ → 1
ದಂ → 2
ವ → 1
ಸು → 1
TOTAL: 12
Line 3: 12

--- SYLLABLE DEBUG ---
ಧಾ → 2
ವ → 1
ಳ → 1
ಯ → 1
ವಿ → 1
ಲೀ → 2
ನ → 1
ವಿ → 1
ಶ → 1
ದ → 1
ವಿ → 1
ಷ → 1
ಯ → 1
ವಿ → 1
ಶೇ → 2
ಷಂ → 2
TOTAL: 20
Line 4: 20

--- RESULT ---
This is a Kanda Padya!
